<a href="https://colab.research.google.com/github/Janamawasy/ML---TreeANN/blob/main/tree_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#importing the dataset
import numpy as np
import pandas as pd
import tensorflow as tf
from datetime import datetime
tf.__version__



'2.15.0'

In [69]:
# import data
dataset = pd.read_csv('Tree_Data.csv')

# data processing :: drop Adult column
dataset = dataset.drop(dataset.columns[8], axis=1)



In [96]:
# Date to Eppoch
# # Convert the given date string to a datetime object
# given_datetime = datetime.strptime('1/31/17', '%m/%d/%y')

# # Convert the datetime object to epoch timestamp (seconds)
# epoch_timestamp_seconds = int(given_datetime.timestamp())

dataset['PlantDate'] = dataset['PlantDate'].apply(lambda x: int(datetime.strptime(x, '%m/%d/%y').timestamp()))

dataset = dataset.dropna(subset=['Event'])


x = dataset.iloc[:, 1:-3]
y = dataset.iloc[:, -3:]



In [97]:
# Label Encoding for the following columns
y_le_features= ['Harvest','Alive']

# Encoding categorial data 'Alive' : Dead = 0 Alive = 1
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
le = LabelEncoder()

y['Harvest'] = le.fit_transform(y['Harvest'])
y['Alive'] = le.fit_transform(y['Alive'])

y['Harvest'] = [bit ^ 1 for bit in y['Harvest']]
y['Alive'] = [bit ^ 1 for bit in y['Alive']]


# Fill NaN vals in'EMF'
column_mean = x['EMF'].mean()
x['EMF'] = x['EMF'].fillna(column_mean)

In [80]:
# # Label Encoding for Light_Cat  - it appears that One Hot Encoding is better option for Light_Cat

# # make sure that the values 0 - low , 1 - mid , 2- high

# # Option 1 : Encoding categorial data 'Light_Cat' : 3 Possible Labels -> [0,1,2]
# # x['Light_Cat'] = le.fit_transform(x['Light_Cat'])

# # Option 2 : Manual Label Encoding :
# # Define a mapping for categories
# category_mapping = {'High': 2, 'Med': 1, 'Low': 0}

# # Use the map function to replace categories with numerical values
# x['Light_Cat'] = x['Light_Cat'].map(category_mapping)



In [98]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

# One Hot Encoding for the follwing columns
ohe_features = ['Subplot','Species','Soil','Sterile','Conspecific','Myco','SoilMyco','Light_Cat']

ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), ohe_features)], remainder='passthrough')
x = np.array(ct.fit_transform(x))


In [99]:
nan_count = np.isnan(x).sum()

print("Number of NaN values:", nan_count)

Number of NaN values: 0


In [100]:
nan_count = np.isnan(y).sum()

print("Number of NaN values:", nan_count)

Number of NaN values: Event      0
Harvest    0
Alive      0
dtype: int64


In [107]:
from sklearn.model_selection import train_test_split

x = np.array(x)
y = np.array(y)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.fit_transform(x_test)

#Training

#Init the ANN
ann = tf.keras.models.Sequential()

ann.add(tf.keras.layers.Dense(units=6, activation='relu'))
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))
ann.add(tf.keras.layers.Dense(units=3, activation='sigmoid'))

ann.compile(optimizer='adam', loss='binary_crossentropy', metrics= ['accuracy'])

# Actual training
ann.fit(x_train, y_train, batch_size=32, epochs=100)


y_pred = ann.predict(x_test)


Epoch 1/100
70/70 [==============================] - 2s 2ms/step - loss: 0.6501 - accuracy: 0.5375
Epoch 2/100
70/70 [==============================] - 0s 2ms/step - loss: 0.5602 - accuracy: 0.5744
Epoch 3/100
70/70 [==============================] - 0s 2ms/step - loss: 0.5009 - accuracy: 0.5744
Epoch 4/100
70/70 [==============================] - 0s 2ms/step - loss: 0.4772 - accuracy: 0.5753
Epoch 5/100
70/70 [==============================] - 0s 2ms/step - loss: 0.4625 - accuracy: 0.5910
Epoch 6/100
70/70 [==============================] - 2s 24ms/step - loss: 0.4476 - accuracy: 0.6715
Epoch 7/100
70/70 [==============================] - 0s 3ms/step - loss: 0.4277 - accuracy: 0.7366
Epoch 8/100
70/70 [==============================] - 0s 2ms/step - loss: 0.4035 - accuracy: 0.7870
Epoch 9/100
70/70 [==============================] - 0s 2ms/step - loss: 0.3812 - accuracy: 0.8117
Epoch 10/100
70/70 [==============================] - 0s 2ms/step - loss: 0.3637 - accuracy: 0.8162
Epoch 11

In [105]:
from sklearn.metrics import multilabel_confusion_matrix,accuracy_score

y_binary_pred = np.where(y_pred > 0.5, 1, 0)

cm = multilabel_confusion_matrix(y_test,y_binary_pred)
acc = accuracy_score(y_test,y_binary_pred)


In [106]:
print(cm)
print('acc is:', acc)

[[[361  13]
  [ 34 427]]

 [[594  32]
  [ 13 196]]

 [[670   0]
  [  1 164]]]
acc is: 0.9425149700598803
